In [1]:
import keras

/Users/perceval/anaconda/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/perceval/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [2]:
import pickle
import os
import re
import time

from keras import backend as K
import tensorflow as tf

import numpy as np
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk.stem import SnowballStemmer
from tqdm import tqdm

In [3]:
np.set_printoptions(linewidth=100, precision=4)

In [318]:
params = {
    "UTTERANCE_LENGTH": 60,
    "DISCUSSION_LENGTH": 48, #60,
    "NARRATION_LENGTH": 30,
    "MAX_WORDS_COUNT": 3000,
    "WORD_EMBEDDING_DIM": 32,
    "PEOPLE_EMBEDDING_DIM": 32,
    "VALIDATION_SPLIT": 0.2,
    "LSTM_UNITS_COUNT": 64,
    "PATIENCE": 2,
    "EPOCHS": 8,
    "BATCH_SIZE": 16, # not large because every sample is 300 words long
    "LR": 0.001,
    "MSE_LOSS_WEIGHT": 500,
}


TIME_STR = time.strftime("%Y%m%d-%H%M%S")
OUT_MODEL_PATH = os.path.join('./output/', "model-{}.h5".format(TIME_STR))
CHECKPOINT_PATH = os.path.join('./output/', "model-{}-checkpoint.h5".format(TIME_STR))
GLOVE_PATH = '/Users/perceval/Developpement/Data/glove.6B.100d/glove.6B.100d.txt'
DATA_PATH = 'corpus/dataset_dl.pkl'
SPLIT_REGEX = '[^a-zA-Z](‘)|(‘)[^a-zA-Z]|(?:\s|\n)+|(\d+\.\d*)|([".!,;:-])'

### Load people

In [319]:
people = pickle.load(open("corpus/people.pkl", "rb"))

In [320]:
dataset = pickle.load(open("corpus/dataset-prolog.pkl", "rb"))

In [321]:
dataset[0][0]

('My dear Mr_Bennet,  have you heard that Netherfield Park is let at last?',
 ' said his lady to him one day, ',
 {'Charlotte', 'Mrs_Bennet'},
 'Mrs_Bennet',
 'wife',
 'F',
 None)

### Load data and process it (again !)

#### Tokenize text

We are going to stem the data, to make the vocabulary denser

In [322]:
def stem_sentence(stemmer, sentence, people_names):
    tokens = re.findall(r"[@_\w]+|['.,!?;]", sentence)
    return [stemmer.stem(word) if word not in people_names else word for word in tokens]

In [323]:
stem_sentence(SnowballStemmer('english'), "Mr_Bennet is with Mrs_Bennet's daughter", [p['main'] for p in people])

['Mr_Bennet', 'is', 'with', 'Mrs_Bennet', "'", 's', 'daughter']

In [324]:
def pad(list, length, default):
    if length is None:
        return list
    return  [list[i] if i < len(list) else default for i in range(length)]

In [325]:
def make_data(dataset, people, max_count=3000, verbose=2, utterance_length=8, narration_length=None, discussion_length=None):
    people_main = [p['main'] for p in people]
    
    stemmer = SnowballStemmer('english')

    stemmed_samples = [
        [(stem_sentence(stemmer, utterance[0], people_main),
          stem_sentence(stemmer, utterance[1], people_main),
          utterance[2],
          utterance[3])
         for utterance in discussion]
        for discussion in (tqdm(dataset, desc="Text words stemming")
                           if verbose > 1 else dataset)]
    
    words = [word
             for discussion in stemmed_samples
             for utterance in discussion
             for text in (utterance[0], utterance[1])
             for word in text]
    # Fit the tokenizer on train texts
    word_index, word_counts = np.unique(words, return_counts=True)
    new_indices = sorted(range(len(word_index)), key=lambda i: "0"+word_index[i] if word_index[i] in people_main else "1"+word_index[i])
    word_index = word_index[new_indices]
    word_counts = word_counts[new_indices]
    
    inverse_words = {v: i+2 for i, v in enumerate(word_index)}
    inverse_people = {v: i for i, v in enumerate(people_main)}

    # Convert them to indices and truncate them if they are too large
    tokenized_samples = [
        # Pad the discussion so that its length matches `discussion_length`
        pad([(pad([inverse_words.get(w, 1) for w in utterance[0]], utterance_length, 0),
              pad([inverse_words.get(w, 1) for w in utterance[1]], narration_length, 0),
              [inverse_people.get(p, 0) for p in utterance[2]],
              inverse_people.get(utterance[3], -1)+1)
              for utterance in discussion],
             length=discussion_length,
             default=([0]*(utterance_length or 0), # empty utterance
                      [0]*(narration_length or 0), # empty narration,
                      [], # no target hint
                      0, # default non-character id
             ))
        for discussion in (tqdm(stemmed_samples, desc="Text/targets to ids mapping")
                           if verbose > 1 else stemmed_samples)]

    return tokenized_samples, word_index

In [326]:
res = make_data(dataset, people,
                max_count=params["MAX_WORDS_COUNT"],
                utterance_length=params["UTTERANCE_LENGTH"],
                narration_length=params["NARRATION_LENGTH"],
                discussion_length=params["DISCUSSION_LENGTH"])
processed_dataset, word_index = res
#res[0][0][0]

Text/targets to ids mapping: 100%|██████████| 94/94 [00:00<00:00, 683.33it/s]


### Shape the dataset as matrices

In [327]:
def make_matrices(data, discussion_length, voc_size, people_count, target_count):
    utterance_matrices = np.zeros((len(data), discussion_length, voc_size)) # Set of words -> Bag of words one-hot encoding
    narration_matrices = np.zeros((len(data), discussion_length, voc_size)) # Set of words -> Bag of words one-hot encoding
    people_matrices = np.zeros((len(data), discussion_length, people_count)) # Set of people -> Bag of words one-hot encoding
    target_matrices = np.zeros((len(data), discussion_length, target_count)) # Categorical target -> One-hot encoding
    for discussion_i, discussion in enumerate(data):
        for utterance_i, utterance in enumerate(discussion):
            utterance_matrices[discussion_i, utterance_i, utterance[0]] = 1
            narration_matrices[discussion_i, utterance_i, utterance[1]] = 1
            people_matrices[discussion_i, utterance_i, list(utterance[2])] = 1
            target_matrices[discussion_i, utterance_i, utterance[3]] = 1
    
    return utterance_matrices, narration_matrices, people_matrices, target_matrices

In [342]:
utterance_matrices, narration_matrices, people_hint_matrices, target_matrices = \
    make_matrices(processed_dataset,
              discussion_length=params['DISCUSSION_LENGTH'],
              voc_size=len(word_index)+2,
              people_count=len(people),
              target_count=len(people)+1)

What does it look like now ? let's look at the first utterance of the first discussion

In [345]:
# display(dataset[0][0])
# display(processed_dataset[0][0])

### Define the model structure

In [346]:
utterances_input_layer = keras.layers.Input(shape=(None, len(word_index)+2)) # leave the dataset length that will be batched
narrations_input_layer = keras.layers.Input(shape=(None, len(word_index)+2)) # leave the dataset length that will be batched
people_hint_input_layer = keras.layers.Input(shape=(None, len(people))) # leave the dataset length that will be batched

#embedding_layer = keras.layers.Embedding(len(word_index), params["WORD_EMBEDDING_DIM"], name="word_embedding")
word_bag_layer = keras.layers.Dense(params["WORD_EMBEDDING_DIM"])#, kernel_regularizer=keras.regularizers.l2(0.01))
people_hint_bag_layer = keras.layers.Dense(params["PEOPLE_EMBEDDING_DIM"])
concat_layer = keras.layers.Concatenate(name="lstm_input")

concat_input = concat_layer([
    word_bag_layer(utterances_input_layer),
    word_bag_layer(narrations_input_layer),
    people_hint_input_layer, #people_hint_bag_layer(people_hint_input_layer),#
])
lstm_input = concat_input#keras.layers.Activation('relu')(concat_input)

# not mandatory to set the LSTM dim output to the people hints dim input but it seems more coherent

lstm_layer = keras.layers.GRU(params['LSTM_UNITS_COUNT'], return_sequences=True)
lstm_output = dropout(lstm_layer(lstm_input))

output_layer = keras.layers.TimeDistributed(keras.layers.Dense(len(people)+1, activation='softmax'))
#lstm_layer = keras.layers.Dense(len(people)+1, activation='softmax')
output = output_layer(lstm_output)

In [347]:
model = keras.Model(inputs = [utterances_input_layer, narrations_input_layer, people_hint_input_layer] , outputs = [output])
model.compile(loss='categorical_crossentropy',
              optimizer='adam', 
              metrics=['accuracy'],
              sample_weight_mode='temporal')

### Sample weighting

Because we have imbalanced classes and padding utterances, we need to weights them to correct the loss function

In [348]:
from sklearn.model_selection import train_test_split

In [349]:
(train_utterance_matrices,   test_utterance_matrices,
 train_narration_matrices,   test_narration_matrices,
 train_people_hint_matrices, test_people_hint_matrices,
 train_target_matrices, test_target_matrices) = \
    train_test_split(utterance_matrices, narration_matrices, people_hint_matrices, target_matrices, test_size=0.2)

In [350]:
# Count the classes
targets_set, targets_inverse, targets_count = np.unique(np.argwhere(train_target_matrices)[:, -1], return_inverse=True, return_counts=True)

# And transform these counts into samples
sample_weight = np.zeros_like(targets_inverse, dtype=float)
sample_weight[targets_inverse == 0] = 0.00
total = sum(targets_count) - targets_count[0]
for target_id, target_count in zip(targets_set, targets_count):
    sample_weight[targets_inverse == target_id] = target_count/total
sample_weight = sample_weight.reshape(train_target_matrices.shape[:2])

In [352]:
sample_weight[np.argwhere(sample_weight.sum(axis=1) == 0).reshape(-1)] += 0.1

In [356]:
model.fit(x=[train_utterance_matrices, train_narration_matrices, train_people_hint_matrices],
          y=[train_target_matrices],
          epochs=50,
          sample_weight=sample_weight,
          validation_split=0.2,
          verbose=1, batch_size=8)
    # print('-----')
    # print(model.evaluate([train_utterance_matrices, train_narration_matrices, train_people_hint_matrices], train_targets_matrices))    
    # print(model.evaluate([test_utterance_matrices, test_narration_matrices, test_people_hint_matrices], test_targets_matrices))

Train on 60 samples, validate on 15 samples
Epoch 1/50
60/60 [==============================] - 7s 121ms/step - loss: 8.5210 - acc: 0.5510 - val_loss: 7.9873 - val_acc: 0.6556
Epoch 2/50
60/60 [==============================] - 1s 15ms/step - loss: 7.5319 - acc: 0.7434 - val_loss: 6.0903 - val_acc: 0.6556
Epoch 3/50
60/60 [==============================] - 1s 19ms/step - loss: 4.2999 - acc: 0.7434 - val_loss: 0.8290 - val_acc: 0.6556
Epoch 4/50
60/60 [==============================] - 1s 16ms/step - loss: 0.3712 - acc: 0.7434 - val_loss: 0.1194 - val_acc: 0.6556
Epoch 5/50
60/60 [==============================] - 1s 16ms/step - loss: 0.1515 - acc: 0.7434 - val_loss: 0.1116 - val_acc: 0.6556
Epoch 6/50
60/60 [==============================] - 1s 16ms/step - loss: 0.1391 - acc: 0.7434 - val_loss: 0.1086 - val_acc: 0.6556
Epoch 7/50
60/60 [==============================] - 1s 16ms/step - loss: 0.1330 - acc: 0.7434 - val_loss: 0.1041 - val_acc: 0.6556
Epoch 8/50
60/60 [====================

In [367]:
model.evaluate(
    x=[test_utterance_matrices[:, :1, :], test_narration_matrices[:, :1, :], test_people_hint_matrices[:, :1, :]],
    y=[test_target_matrices[:, :1, :]])

19/19 [==============================] - 0s 340us/step


[5.182927131652832, 0.21052631735801697]

In [368]:
from sklearn.metrics import accuracy_score

In [377]:
model.predict(x=[test_utterance_matrices, test_narration_matrices, test_people_hint_matrices]).reshape(-1, test_target_matrices.shape[-1]).shape

(912, 55)

In [379]:
np.argwhere(test_target_matrices.reshape(-1, test_target_matrices.shape[-1]))[:, 1].shape

(912,)

In [382]:
accuracy_score(
    model.predict(x=[test_utterance_matrices, test_narration_matrices, test_people_hint_matrices]).reshape(-1, test_target_matrices.shape[-1]),
    test_target_matrices.reshape(-1, test_target_matrices.shape[-1])
)

ValueError: Classification metrics can't handle a mix of continuous-multioutput and multilabel-indicator targets

In [167]:
model.fit(x=[utterance_matrices, people_hint_matrices],
          y=[target_matrices],
          epochs=150,
          sample_weight=sample_weight)

Epoch 1/150
94/94 [==============================] - 0s 1ms/step - loss: 0.0864 - acc: 0.3604
Epoch 2/150
94/94 [==============================] - 0s 1ms/step - loss: 0.0875 - acc: 0.3484
Epoch 3/150
94/94 [==============================] - 0s 1ms/step - loss: 0.0863 - acc: 0.3644
Epoch 4/150
94/94 [==============================] - 0s 1ms/step - loss: 0.0901 - acc: 0.3457
Epoch 5/150
94/94 [==============================] - 0s 1ms/step - loss: 0.0851 - acc: 0.3511
Epoch 6/150
94/94 [==============================] - 0s 1ms/step - loss: 0.0844 - acc: 0.3497
Epoch 7/150
94/94 [==============================] - 0s 961us/step - loss: 0.0850 - acc: 0.3577
Epoch 8/150
94/94 [==============================] - 0s 943us/step - loss: 0.0864 - acc: 0.3537
Epoch 9/150
94/94 [==============================] - 0s 963us/step - loss: 0.0875 - acc: 0.3391
Epoch 10/150
94/94 [==============================] - 0s 946us/step - loss: 0.0874 - acc: 0.3457
Epoch 11/150
94/94 [==============================]

In [170]:
target_matrices[0]

array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 

### Design fake data to force the model to pay attention to the hints

In [62]:
fake_mono_discussion_count = len(people)#+1
fake_utterance_length = 40
fake_mono_utterance_matrices = np.zeros((fake_mono_discussion_count, 1, len(word_index)+2))
fake_mono_narration_matrices = np.zeros((fake_mono_discussion_count, 1, len(word_index)+2))

fake_mono_people_hint_matrices = np.zeros((fake_mono_discussion_count, 1, len(people)))
fake_mono_people_hint_matrices[np.arange(len(people)), 0, np.arange(len(people))] = 1
fake_mono_target_matrices = np.zeros((fake_mono_discussion_count, 1, len(people)+1))
fake_mono_target_matrices[np.arange(len(people)), 0, np.arange(len(people))] = 1

In [63]:
fake_mono_utterance_matrices = fake_mono_utterance_matrices.repeat(50, axis=0)
fake_mono_narration_matrices = fake_mono_narration_matrices.repeat(50, axis=0)
fake_mono_people_hint_matrices = fake_mono_people_hint_matrices.repeat(50, axis=0)
fake_mono_target_matrices = fake_mono_target_matrices.repeat(50, axis=0)

In [64]:
for i in range(fake_mono_utterance_matrices.shape[0]):
    for j in range(fake_mono_utterance_matrices.shape[1]):
        fake_mono_utterance_matrices[i][j][np.random.choice(fake_mono_utterance_matrices.shape[2], fake_utterance_length)] = 1

In [426]:
model.fit(
    x=[fake_mono_utterance_matrices, fake_mono_people_hint_matrices],
    y=[fake_mono_target_matrices],
    verbose=1,
    epochs=5,
)

Epoch 1/5
2700/2700 [==============================] - 2s 689us/step - loss: 3.3883 - acc: 0.2356
Epoch 2/5
2700/2700 [==============================] - 2s 652us/step - loss: 3.3707 - acc: 0.2374
Epoch 3/5
2700/2700 [==============================] - 2s 736us/step - loss: 3.3606 - acc: 0.2437
Epoch 4/5
2700/2700 [==============================] - 2s 662us/step - loss: 3.3515 - acc: 0.2556
Epoch 5/5
2700/2700 [==============================] - 2s 702us/step - loss: 3.3436 - acc: 0.2652


In [68]:
#display(fake_mono_target_matrices[[0, 50]])
model.evaluate([fake_mono_utterance_matrices, fake_mono_people_hint_matrices], [fake_mono_target_matrices])

2700/2700 [==============================] - 1s 198us/step


[0.4065698538886176, 1.0]

### Design fake data to force the model to pay attention to alternative speakers

In [164]:
fake_people_pairs = np.array([np.random.choice(len(people), 2) for _ in range(500)])

fake_duo_discussion_count = len(fake_people_pairs)
fake_duo_discussion_length = 32

fake_duo_utterance_matrices = np.zeros((fake_duo_discussion_count, fake_duo_discussion_length, len(word_index)+2))
fake_duo_narration_matrices = np.zeros((fake_duo_discussion_count, fake_duo_discussion_length, len(word_index)+2))

fake_duo_people_hint_matrices = np.zeros((fake_duo_discussion_count, fake_duo_discussion_length, len(people)))
# get a hint in the first utterance about the first speaker
fake_duo_people_hint_matrices[np.arange(fake_duo_discussion_count), 0, fake_people_pairs[:, 0]] = 1
# get a hint in the second utterance about the second speaker
fake_duo_people_hint_matrices[np.arange(fake_duo_discussion_count), 1, fake_people_pairs[:, 1]] = 1

for i in range(fake_duo_utterance_matrices.shape[0]):
    for j in range(fake_duo_utterance_matrices.shape[1]):
        fake_duo_utterance_matrices[i][j][np.random.choice(fake_duo_utterance_matrices.shape[2], fake_utterance_length)] = 1
        
fake_duo_target_matrices = np.zeros((fake_duo_discussion_count, fake_duo_discussion_length, len(people)+1))
for i in range(fake_duo_discussion_length//2):
    fake_duo_target_matrices[np.arange(fake_duo_discussion_count), i*2, fake_people_pairs[:, 0]] = 1
    fake_duo_target_matrices[np.arange(fake_duo_discussion_count), i*2+1, fake_people_pairs[:, 1]] = 1

In [165]:
for i in range(5):
    model.fit(
        x=[fake_duo_utterance_matrices, fake_duo_people_hint_matrices],
        y=[fake_duo_target_matrices],
        verbose=0,
        epochs=50
    )
    print(model.evaluate([fake_duo_utterance_matrices, fake_duo_people_hint_matrices], [fake_duo_target_matrices]))

500/500 [==============================] - 2s 4ms/step
[2.1751780014038085, 0.45024999952316286]
500/500 [==============================] - 1s 2ms/step
[1.1733175048828124, 0.5223750009536743]
500/500 [==============================] - 1s 2ms/step
[0.8949550933837891, 0.5454375]
500/500 [==============================] - 1s 2ms/step
[0.7860103082656861, 0.6003750004768371]
500/500 [==============================] - 1s 2ms/step
[0.44410645246505737, 0.9123749995231628]


In [51]:
display(fake_duo_target_matrices[0, :2])
model.predict([fake_duo_utterance_matrices[:1], fake_duo_people_hint_matrices[:1]])

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

array([[[1.0683e-02, 1.5623e-02, 1.6022e-02, ..., 2.1181e-02, 2.0678e-02, 3.2961e-03],
        [2.1510e-03, 9.1679e-03, 5.4895e-03, ..., 1.9287e-02, 2.1664e-02, 4.3625e-04],
        [1.7540e-04, 2.3571e-03, 6.7826e-04, ..., 5.5822e-03, 1.1317e-02, 4.1458e-05],
        ...,
        [1.5415e-06, 3.9914e-04, 8.1925e-06, ..., 2.7226e-04, 5.8026e-03, 5.8685e-06],
        [2.0408e-06, 4.4793e-04, 9.4922e-06, ..., 3.1717e-04, 6.1624e-03, 6.8599e-06],
        [1.6265e-06, 4.0058e-04, 8.4284e-06, ..., 2.7779e-04, 5.8704e-03, 5.7684e-06]]],
      dtype=float32)

In [974]:
nb_discussion = discussion_int
nb_replique_max = l_max
nombre_mots = longueur
embedding_incise_dim = nb_personnages+1

#EMBEDDING
embedding_output_dim = 1

input_text = keras.layers.Input(shape =(nb_replique_max, nombre_mots,), name = "main")
embedding_layer = keras.layers.Embedding(nombre_mots, embedding_output_dim, input_length = None,
                                        embeddings_initializer='uniform',embeddings_regularizer=None, 
                                        activity_regularizer=None,embeddings_constraint=None,
                                        mask_zero=False)(input_text)


#BAG_OF_WORDS
bag = keras.layers.Lambda(lambda x: K.sum(x, axis=2), output_shape=(nb_replique_max, embedding_output_dim,))(embedding_layer)


#INPUTBIS
input_incise = keras.layers.Input((nb_replique_max, embedding_incise_dim, ), name = "aux")


#MERGING
merged = keras.layers.concatenate([bag, input_incise])


#LSTM
units = nb_personnages + 1
#lstm = keras.layers.LSTM(units, return_sequences=True) (merged)
lstm = keras.layers.LSTM(units, return_sequences=True) (input_incise)

#MODEL

sample = np.array([[int(col != 0) for col in lig] for lig in people])

#OUT
out = keras.layers.Activation('softmax') (lstm)

model = keras.Model(inputs = [input_text, input_incise] , outputs = out)
#model = keras.Model(inputs = input_incise, outputs = out)
sgd = keras.optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.5, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='sgd', 
              metrics=['accuracy'], sample_weight_mode='temporal')


#data_zero = np.zeros((discussion_int, l_max, longueur))
x_train = [data,Z]
y_train = Y
model.fit(x_train, y_train,
          epochs=20,
          batch_size=1,
          sample_weight = sample)

<tf.Tensor 'input_2:0' shape=(?, 200) dtype=float32>